In [14]:
import pandas as pd
import numpy as np
import time

In [15]:
def api_url(params):
    '''
    Helper to create request string
    '''
    API_ENDPOINT = 'https://comtrade.un.org/api/get?'
    
    return API_ENDPOINT + '&'.join(
        [ '{k}={v}'.format(k=k, v=v) for k, v in params.items()])

In [16]:
params_trade_total = {
    'max': 1000,
    'type': 'C',
    'freq': 'A',
    'px': 'HS',
    'ps': 2020,
    'r': 643,    # Russia's code
    'p': 'all',
    'rg': 'all',
    'cc': 'TOTAL',
    'fmt': 'csv'
}

### RUSSIAN IMPORT/EXPORT TRADE VALUE PER PARTNER: 2000 - 2020

In [ ]:
params_trade_total['fmt'] = 'csv'

for year in range(2000, 2021):
    params_trade_total['ps'] = year
    
    data = pd.read_csv(
        api_url(params_trade_total)
    )
    print('Year:', year, 'records:', len(data))
    
    data.to_csv(
        'data/total-trade-value-{year}.csv'.format(year=year),
        index=False
    )
    time.sleep(30)

### !TODO: IMPORT/EXPORT PRODUCTS FROM TOP 10 PARTNERS

In [ ]:
params_products = {
    'max': 5000,
    'type': 'C',
    'freq': 'A',
    'px': 'HS',
    'ps': 2020,
    'r': 643,    # Russia's code
    'p': [156, 276],   # Partner code separated by comma
    'rg': [1, 2],    # 1 - Import, 2 - Export
    'cc': 'AG2',
    'fmt': 'csv'
}

In [37]:
top_partners_stats = pd.read_csv('data/top_partners_stats.csv')

In [38]:
top_partners_stats

,Year,Trade Flow,Rank,index,Partner,Partner Code,Partner Pct,Trade Value (US$)
0,2000,Export,1,240,Germany,276,8.95,9231828795
1,2000,Export,2,257,Italy,381,7.04,7254325411
2,2000,Export,3,206,Belarus,112,5.40,5572101000
3,2000,Export,4,215,China,156,5.09,5247619974
4,2000,Export,5,343,Ukraine,804,4.87,5024485125
5,2000,Export,6,346,United Kingdom,826,4.53,4670041638
6,2000,Export,7,348,USA,842,4.51,4648015624
7,2000,Export,8,306,Poland,616,4.32,4452274283
8,2000,Export,9,293,Netherlands,528,4.22,4349439032
9,2000,Export,10,331,Switzerland,757,3.86,3976663636
